In [1]:
#Borrar entorno
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr,gt)
showtext_auto()

Cargando paquete requerido: pacman



In [2]:
url_basica<- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<- "microdatos/enigh"
archivos<- c("ns_concentradohogar","ns_gastospersona","ns_gastoshogar")
years<- c("2024")
options(survey.lonely.psu = "certainty") 

In [3]:
# Crear carpeta 'data'
if(!dir.exists("data")) dir.create("data")  
options(timeout = 600)

In [4]:
for (archivo in archivos) {
  for (i in seq_along(years)) {

    url      <- paste0(url_basica, years[i], "/", segunda_parte, years[i], "_", archivo, "_csv.zip")
    destfile <- paste0("data/", years[i], "_", archivo, ".zip")

    message("Descargando: ", url)
    download.file(url, destfile = destfile, mode = "wb")

    # Descomprimir y capturar rutas extraídas 
    extraidos <- unzip(destfile, exdir = "data", junkpaths = TRUE)

    # Elegir el CSV correcto según el tipo ("concentradohogar" o "gastospersona")
    needle   <- sub("^ns_", "", archivo)
    csv_path <- extraidos[
      grepl("\\.csv$", extraidos, ignore.case = TRUE) &
      grepl(needle, basename(extraidos), ignore.case = TRUE)
    ][1]

    # Renombrar con año
    nuevo <- file.path("data", paste0(archivo, "_", years[i], ".csv"))
    if (!is.na(csv_path) && file.exists(csv_path)) {
      file.rename(csv_path, nuevo)
      message("✅ Guardado: ", nuevo)
    } else {
      warning("No se encontró el CSV para ", archivo, " en ", years[i])
    }

    # Borrar zip
    file.remove(destfile)
  }
}

Descargando: https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_concentradohogar_csv.zip

✅ Guardado: data/ns_concentradohogar_2024.csv

Descargando: https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_gastospersona_csv.zip

✅ Guardado: data/ns_gastospersona_2024.csv

Descargando: https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_gastoshogar_csv.zip

✅ Guardado: data/ns_gastoshogar_2024.csv



In [5]:
# Rutas
path_hogares      <- "data/ns_concentradohogar_2024.csv"
path_gastohogar   <- "data/ns_gastoshogar_2024.csv"
path_gastopersona <- "data/ns_gastospersona_2024.csv"

# Claves de transporte público
claves_tp <- c(
  "073215"
) 

In [6]:
# Hogares y creación de quintiles ====
hogares <- read_csv(path_hogares, show_col_types = FALSE) %>%
  arrange(ing_cor, folioviv, foliohog) %>%
  #Crear claves de entidad
      mutate(
      folioviv = as.character(folioviv),
      foliohog = as.character(foliohog),
      cve_ent = case_when(
        nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
        nchar(folioviv) == 10 ~ substr(folioviv, 1, 2),
        TRUE ~ NA_character_
      )
    )%>%
      filter(cve_ent=="09") %>%
  mutate(
    c0 = lag(cumsum(factor), default = 0),
    c1 = c0 + factor
  ) %>%
  {
    tam_dec <- floor(sum(.$factor, na.rm = TRUE) / 5)

    rowwise(.) %>%
      mutate(.split = list({
        d_ini <- pmax(1L, floor(c0 / tam_dec) + 1L)
        k <- d_ini:5L
        left  <- (k - 1L) * tam_dec
        right <- ifelse(k == 5L, Inf, k * tam_dec)

        tibble(
          quintil_hogar = k,
          factor_part = pmax(0, pmin(c1, right) - pmax(c0, left))
        ) %>% filter(factor_part > 0)
      })) %>%
      ungroup() %>%
      unnest(.split) %>%
      mutate(factor = factor_part) %>%
      select(-factor_part, -c0, -c1)
  }


In [7]:
# Función para leer gasto transporte público
leer_gasto_tp <- function(path) {
  read_csv(path, show_col_types = FALSE) %>%
    filter(clave %in% claves_tp) %>%
          mutate(
      folioviv = as.character(folioviv),
      foliohog = as.character(foliohog)) %>%
    select(folioviv, foliohog, gasto_tri) %>%
    group_by(folioviv, foliohog) %>%
    summarise(gasto_tp = sum(gasto_tri, na.rm = TRUE), .groups = "drop")
}

In [8]:
#Gasto hogar y persona
gasto_hogar_tp <- leer_gasto_tp(path_gastohogar) %>%
  rename(gasto_transporte_hogar = gasto_tp)

gastos_persona_tp <- leer_gasto_tp(path_gastopersona) %>%
  rename(gasto_transporte_persona = gasto_tp)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [9]:
# 4) Pegar a hogares
hogares_final <- hogares %>%
  left_join(gastos_persona_tp, by = c("folioviv", "foliohog")) %>%
  left_join(gasto_hogar_tp,   by = c("folioviv", "foliohog")) %>%
  mutate(
    across(
      c(gasto_transporte_persona, gasto_transporte_hogar),
      ~ coalesce(.x, 0)
    ),
    gasto_transporte_total = gasto_transporte_persona + gasto_transporte_hogar
  )%>%
  #Identficar hogares que usan microbús o autobús urbano
  mutate(dummy_microbus=case_when(gasto_transporte_total>0 ~ 1,
                                    TRUE ~ 0))

In [10]:
dm <- hogares_final %>%
    as_survey_design(ids = upm, strata = est_dis, weights = factor, nest = TRUE)

In [11]:
###Hogares en donde al menos un integrante utiliza microbús o autobús urbano
hog_micro<-dm %>%
  group_by(dummy_microbus)%>%
  summarise(
    hogares=survey_total(vartype="cv"),
    pct_hogares = survey_mean(vartype="cv")
  )%>%
  mutate(pct_hogares = pct_hogares*100)

  hog_micro

dummy_microbus,hogares,hogares_cv,pct_hogares,pct_hogares_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2083100,0.02561263,67.58199,0.02183375
1,999230,0.04710226,32.41801,0.04551693


In [12]:
hog_micro%>%
select(dummy_microbus, hogares, pct_hogares)%>%
mutate(
    dummy_microbus = case_when(
      dummy_microbus == 1 ~ "Hogares que usan transporte colectivo, combi o microbús",
      dummy_microbus == 0 ~ "Hogares que no usan transporte colectivo, combi o microbús"
    )
  )%>%
  janitor::adorn_totals("row")%>%
gt()%>%
  fmt_number(
    columns = c(hogares),
    decimals = 0,
    use_seps = TRUE
  )%>%
  fmt_number(
    columns = c(pct_hogares),
    decimals = 2
  )%>%
  cols_label(
    dummy_microbus = md("Condición del hogar"),
    hogares = "Número de hogares",
    pct_hogares = "Porcentaje de hogares (%)"
  )%>%
  tab_header(
    title = md("**Hogares en la Ciudad de México que destinan gasto a servicio de transporte colectivo, combi o microbús**"
  ))%>%
  tab_source_note(
    source_note = "Fuente: @claudiodanielpc con datos de INEGI. Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) 2024."
  )%>%
    gtsave("hogares_microbus_cdmx.png") 

In [13]:
#Calcular gasto en microbús respecto del total del ingreso y del total de la canasta de transporte
dm%>%
#Filtrar microbús
filter(dummy_microbus==1)%>%
  summarise(
    hogares=survey_total( vartype="cv"),
    gasto_tp=survey_total(publico  ,vartype="cv"),
    gasto_micro=survey_total(gasto_transporte_total,vartype="cv"),
    pct_canasta_transporte=survey_ratio(gasto_transporte_total, publico, vartype="cv")
  )%>%
    mutate(
        pct_canasta_transporte= pct_canasta_transporte*100
        )


hogares,hogares_cv,gasto_tp,gasto_tp_cv,gasto_micro,gasto_micro_cv,pct_canasta_transporte,pct_canasta_transporte_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
999230,0.04710226,3347536849,0.0679346,1582488881,0.067836,47.27323,0.04400691


In [14]:
##Proporción por quintil de hogar

quintiles<-dm%>%
#Filtrar hogares en donde al menos un integrante utiliza microbús o autobús urbano
filter( dummy_microbus==1)%>%
  group_by(quintil_hogar)%>%
  summarise(
    hogares=survey_total( vartype="cv"),
    ing_prom=survey_mean(ing_cor, vartype="cv"),
    gasto_transporte_prom=survey_mean(publico, vartype="cv"),
    gasto_micro_prom=survey_mean(gasto_transporte_total, vartype="cv"),
    ratio=survey_ratio(gasto_transporte_total, ing_cor, vartype="cv"),
    pct_canasta_transporte=(survey_ratio(gasto_transporte_total, publico, vartype="cv")))%>%
    mutate(pct_canasta_transporte= pct_canasta_transporte*100)
quintiles

quintil_hogar,hogares,hogares_cv,ing_prom,ing_prom_cv,gasto_transporte_prom,gasto_transporte_prom_cv,gasto_micro_prom,gasto_micro_prom_cv,ratio,ratio_cv,pct_canasta_transporte,pct_canasta_transporte_cv
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,227548,0.08685295,33247.67,0.021332852,2204.128,0.06628832,1150.161,0.06809600,0.034593744,0.06636179,52.18214,0.06440293
2,234869,0.09112696,57297.40,0.008497409,3496.937,0.11337744,1739.249,0.07276500,0.030354767,0.07146450,49.73636,0.12580530
3,240376,0.08547815,83088.31,0.008119503,3202.118,0.06567907,1667.704,0.08358920,0.020071466,0.08487851,52.08129,0.05596371
4,179438,0.09159953,121378.96,0.011874025,3743.416,0.08275406,1581.437,0.08288511,0.013028923,0.08580674,42.24583,0.08007634
5,116999,0.12430062,205267.12,0.029649967,4985.054,0.13423848,1945.574,0.15329786,0.009478256,0.15714040,39.02815,0.15734186


In [15]:
#Tabla general

quintiles%>%
select(quintil_hogar, hogares, ing_prom, gasto_transporte_prom, gasto_micro_prom, ratio, pct_canasta_transporte)%>%
mutate(
    quintil_hogar = case_when(
        quintil_hogar == 1 ~ "I",
        quintil_hogar == 2 ~ "II",
        quintil_hogar == 3 ~ "III",
        quintil_hogar == 4 ~ "IV",
        quintil_hogar == 5 ~ "V"
    )
    )%>%
    gt()%>%
    fmt_number(
    columns = c(hogares),
    decimals = 0,
    use_seps = TRUE
    )%>%
    fmt_number(
    columns =
    c(ing_prom, gasto_transporte_prom, gasto_micro_prom),
    decimals = 2,
    use_seps = TRUE
    )%>%
    fmt_number(
    columns = c(ratio),
    decimals = 4
    )%>%
    fmt_number(
    columns = c(pct_canasta_transporte),
    decimals = 2
    )%>%
    cols_label(
    quintil_hogar = md("**Quintil de hogar**"),
    hogares = md("**Número de hogares**"),
    ing_prom = md("**Ingreso corriente promedio (MXN)**"),
    gasto_transporte_prom = md("**Gasto promedio en transporte público (MXN)**
    "),
    gasto_micro_prom = md("**Gasto promedio en servicio de transporte colectivo, combi o microbús (MXN)**"),
    ratio = md("**Promedio de gasto por cada peso de ingreso corriente**"),
    pct_canasta_transporte = md("**Porcentaje del gasto en servicio de transporte colectivo, combi o microbús respecto del gasto en transporte público**")
    )%>%
    tab_header(
    title = md("**Datos generales de gasto en servicio de transporte colectivo, combi o microbús por quintil de hogar en la Ciudad de México**")
    )%>%
    tab_source_note(
    source_note = "Fuente: @claudiodanielpc con datos de INEGI. Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) 2024."
    )%>%
    gtsave("quintiles_microbus_cdmx.png")

